# Mood prediction with TensorFlow
Trying to predict the mood of a user each day with:
- Mood rated between 0 and 5
- Weather rated between 0 and 5
- Temperature
- Month of the year
- Day of the week between 0 and 6

In [14]:
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input

import plotly.express as px

In [15]:
mood_history = pd.read_csv('mood-history.csv').sort_values('timestamp')
mood_history.head()

,timestamp,mood,weather,temperature,month,day,weekday,outside
0,1615935641,3,3,5,3,17,2,1
1,1616022041,2,2,4,3,18,3,0
2,1616108441,2,2,5,3,19,4,0
3,1616194841,4,4,7,3,20,5,1
4,1616320024,5,5,8,3,21,6,1


In [16]:
fig = px.scatter(
                 mood_history,
                 x='day',
                 y='mood',
                 size='weather',
                 color='temperature',
                 title='Correlation between daily mood (y) and weather (point size) and temperature (point color)'
)
fig.show()

Unsupported

In [17]:
mood_history.groupby(['weekday']).mean()

,timestamp,mood,weather,temperature,month,day,outside
weekday,,,,,,,
0,1.616702e+09,4.000000,4.500000,14.000000,3.000000,25.500000,0.500000
1,1.616835e+09,3.500000,5.000000,13.500000,3.000000,26.500000,0.000000
2,1.616535e+09,3.666667,4.333333,12.000000,3.000000,24.000000,0.666667
3,1.616658e+09,2.666667,3.333333,9.000000,3.333333,14.666667,0.000000
4,1.616745e+09,3.000000,3.333333,9.666667,3.333333,15.666667,0.333333
5,1.616833e+09,4.333333,4.666667,8.666667,3.333333,16.666667,1.000000
6,1.616629e+09,5.000000,5.000000,11.500000,3.000000,24.500000,1.000000


In [18]:
fig = px.bar(
             mood_history.groupby(['weekday']).mean(), 
             x=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'], 
             y='mood',
             title="Mean of moods per week day",
             labels=dict(x="Day of the week")
)
fig.show()

Unsupported

## Data normalization

Normalizes some input features to make them all between 0 and 1.

Applies `(x - min) / (max - min)` for all the features.

In [19]:
def init_bag(size, index):
    """
    Initialize a bag of a specified size with an index in which the value
    will be 1.
    """

    bag = [0] * size
    bag[index] = 1

    return bag

def normalize_data(df):
    """
    Normalize the given Data Frame to put all values between 0 and 1.
    Creates bags for the months, days and week days.
    """

    if "mood" in df.columns:
        df['mood'] = df['mood'].apply(lambda x: x / 5)
    df['weather'] = df['weather'].apply(lambda x: x / 5)
    df['temperature'] = df['temperature'].apply(lambda x: (x + 30) / (50 + 30))

    df['month'] = df['month'].apply(lambda x: init_bag(12, x))
    df['day'] = df['day'].apply(lambda x: init_bag(32, x))
    df['weekday'] = df['weekday'].apply(lambda x: init_bag(7, x))

    return df

normalized_mood_history = normalize_data(mood_history)
normalized_mood_history

,timestamp,mood,weather,temperature,month,day,weekday,outside
0,1615935641,0.6,0.6,0.4375,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0]",1
1,1616022041,0.4,0.4,0.4250,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0]",0
2,1616108441,0.4,0.4,0.4375,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0]",0
3,1616194841,0.8,0.8,0.4625,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0]",1
4,1616320024,1.0,1.0,0.4750,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1]",1
5,1616406424,0.6,0.8,0.4750,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0]",0
6,1616579224,0.6,1.0,0.4875,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0]",0
7,1616579224,0.6,1.0,0.4875,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0]",0
8,1616665624,0.4,0.8,0.4625,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0]",0
9,1616752024,0.4,0.6,0.4375,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0]",0


In [20]:
def get_training_data(df):
    """
    Flatten, removes columns names for integers to get the training data.
    """

    train = df[[
                'weather',
                'temperature',
                'month',
                'day',
                'weekday',
                'outside'
    ]]
    # Create new columns as bags for months, days and weekday
    train[[f'm{i}' for i in range(0, 12)]] = train['month'].apply(pd.Series)
    train[[f'd{i}' for i in range(0, 32)]] = train['day'].apply(pd.Series)
    train[[f'w{i}' for i in range(0, 7)]] = train['weekday'].apply(pd.Series)

    # Drop the old columns
    train.drop(['month', 'day', 'weekday'], inplace=True, axis=1)

    # Remove columns names for integers
    train.columns = range(train.shape[1])

    return train

X_train = get_training_data(normalized_mood_history)
X_train.head()

/opt/anaconda3/envs/datalore-user/lib/python3.7/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,0.6,0.4375,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0.4,0.4250,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0.4,0.4375,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.8,0.4625,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1.0,0.4750,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
Y_train = mood_history[[
            'mood'
]]
Y_train.columns = range(Y_train.shape[1])

Y_train.head()

,0
0,0.6
1,0.4
2,0.4
3,0.8
4,1.0


In [22]:
model = Sequential()

model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(1))

In [23]:
model.compile(optimizer='sgd', loss='mean_squared_error')

In [27]:
model.fit(X_train, Y_train, batch_size=1, epochs=100)

Epoch 1/100
18/18 [==============================] - 0s 1ms/step - loss: 0.0441
Epoch 2/100
18/18 [==============================] - 0s 1ms/step - loss: 0.0464
Epoch 3/100
18/18 [==============================] - 0s 1ms/step - loss: 0.0444
Epoch 4/100
18/18 [==============================] - 0s 1ms/step - loss: 0.0434
Epoch 5/100
18/18 [==============================] - 0s 1ms/step - loss: 0.0427
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: 0.0410
Epoch 7/100
18/18 [==============================] - 0s 1ms/step - loss: 0.0375
Epoch 8/100
18/18 [==============================] - 0s 1ms/step - loss: 0.0404
Epoch 9/100
18/18 [==============================] - 0s 1ms/step - loss: 0.0399
Epoch 10/100
18/18 [==============================] - 0s 2ms/step - loss: 0.0390
Epoch 11/100
18/18 [==============================] - 0s 1ms/step - loss: 0.0377
Epoch 12/100
18/18 [==============================] - 0s 1ms/step - loss: 0.0378
Epoch 13/100
18/18 [=================

In [32]:
future_data = pd.DataFrame([[
            5,
            24,
            4,
            4,
            6,
            1
]], columns=['weather', 'temperature', 'month', 'day', 'weekday', 'outside'])
future_data

,weather,temperature,month,day,weekday,outside
0,5,24,4,4,6,1


In [33]:
future_data = normalize_data(future_data)
future_data = get_training_data(future_data)

predictions = model.predict(future_data)

predictions

array([[1.0339991]], dtype=float32)